In [1]:
from iotbx.data_manager import DataManager
from cctbx.array_family import flex
import numpy as np
from qscore_utils import (
          sphere_points,
          #stack_fill,
          trilinear_interpolation,
          rowwise_corrcoef,
          )
from tqdm.notebook import tqdm
from multiprocessing import Pool
import math
import cctbx
from cctbx.array_family import flex
from scitbx_array_family_flex_ext import mean as flex_mean
from scitbx_array_family_flex_ext import max as flex_max
from scitbx_array_family_flex_ext import min as flex_min

def flex_std(array):
    return array.standard_deviation_of_the_sample()

In [2]:
from qscore import radial_shell_mp, radial_shell_worker

In [3]:
def get_depth(lst, level=1):
    """Recursively determine the depth of the nested list"""
    if not isinstance(lst, list) or not lst:
        return level
    return max(get_depth(item, level + 1) for item in lst)

def flatten_nested_list(nested_list, target_dim=None):
    current_depth = get_depth(nested_list)
    
    if target_dim is None:
        target_dim = 1
        
    if target_dim <= 0 or target_dim > current_depth:
        raise ValueError(f"Target dimension must be between 1 and {current_depth} (inclusive).")

    flatten_count = current_depth - target_dim
    flattened_list = nested_list
    for _ in range(flatten_count):
        flattened_list = [item for sublist in flattened_list for item in (sublist if isinstance(sublist, list) else [sublist])]
    return flattened_list



In [4]:
def sphere_points(ctr, rad, N):
    if ctr.ndim==1:
        ctr = ctr[None,:]
    h = -1.0 + (2.0 * np.arange(N) / float(N-1))[:, np.newaxis]
    phis = np.arccos(h)
    thetas = np.zeros_like(phis)
    thetas[1:-1, :] = (3.6 / np.sqrt(N * (1.0 - h[1:-1]**2))) % (2 * np.pi)
    thetas = np.cumsum(thetas, axis=0)

    x = np.sin(phis) * np.cos(thetas)
    y = np.sin(phis) * np.sin(thetas)
    z = np.cos(phis)

    # Stack x, y, z to form points and multiply by rad
    points = rad * np.stack([x, y, z], axis=-1)

    # Reshape points to (1, N, 3)
    points = points.reshape(1, N, 3)

    # Add center coordinates to all points
    # ctr shape: (M, 3), points shape: (1, N, 3)
    # Resultant shape: (M, N, 3)
    pts = ctr[:, np.newaxis, :] + points

    return pts

def sphere_points_flex(ctr, rad, N):
    assert len(ctr.focus())==2 and ctr.focus()[1]==3, "Provide an array of cartesian coordinates, shape (N,3)"
    h = -1.0 + (2.0 * np.arange(N) / float(N-1))[:, np.newaxis]
    phis = np.arccos(h)
    thetas = np.zeros_like(phis)
    thetas[1:-1, :] = (3.6 / np.sqrt(N * (1.0 - h[1:-1]**2))) #% (2 * np.pi)
    thetas = np.cumsum(thetas, axis=0)

    x = np.sin(phis) * np.cos(thetas)
    y = np.sin(phis) * np.sin(thetas)
    z = np.cos(phis)

    # Stack x, y, z to form points and multiply by rad
    points = rad * np.stack([x, y, z], axis=-1)

    # Reshape points to (1, N, 3)
    points = points.reshape(1, N, 3)

    # Add center coordinates to all points
    # ctr shape: (M, 3), points shape: (1, N, 3)
    # Resultant shape: (M, N, 3)
    pts = ctr[:, np.newaxis, :] + points

    return pts
    

In [5]:


def floorf_flex(value, divisor):
    # Assuming value is a flex array and divisor is a scalar
    if divisor == 0.0:
        raise ValueError("Cannot divide by zero!")

    quotient = value / divisor

    # Getting boolean selectors
    pos_sel = quotient >= 0
    neg_sel = ~pos_sel 

    # Applying floor for positive values and ceil for negative values
    quotient.set_selected(pos_sel, flex.floor(quotient.select(pos_sel)))
    quotient.set_selected(neg_sel, flex.ceil(quotient.select(neg_sel)) - 1)

    return quotient


    
def mod_flex(value, divisor):
    # Getting boolean selectors
    pos_sel = value >= 0
    neg_sel = ~pos_sel 
    pos_val = value.select(pos_sel)
    neg_val = value.select(neg_sel)
    pos_quotient = floorf_flex(pos_val,divisor)
    pos_out = pos_val-pos_quotient * divisor
    neg_quotient = floorf_flex(neg_val,divisor)-1
    neg_out = neg_val-neg_quotient * divisor
    
    out = flex.double(len(value),0.0)
    out = out.set_selected(pos_sel,pos_out)
    out = out.set_selected(neg_sel,neg_out)
    return out

In [6]:
def sphere_points_combined(ctr_np,ctr_flex,rad,N):

    h_np = -1.0 + (2.0 * np.arange(N) / float(N-1))[:, np.newaxis]
    h_flex = -1.0 + (2.0 * flex.double_range(N) / (N-1))
    #h_flex.reshape(flex.grid(N,1))
    assert np.all(np.isclose(h_np.flatten(),h_flex.as_numpy_array()))
    phis_flex = flex.acos(h_flex)
    phis_np = np.arccos(h_np)

    thetas_np = np.zeros_like(phis_np)
    thetas_flex = flex.double(len(phis_flex),0.0)
    a_flex = (3.6 / flex.sqrt(N * (1.0 - h_flex[1:-1]**2)))
    a_np = (3.6 / np.sqrt(N * (1.0 - h_np[1:-1]**2)))
    assert np.all(np.isclose(a_np.flatten(),a_flex.as_numpy_array()))

    thetas_np[1:-1, :] = a_np
    thetas_flex = thetas_flex.set_selected(flex.uint32_range(1,N-1),a_flex)
    assert np.all(np.isclose(thetas_np.flatten(),thetas_flex.as_numpy_array()))


    thetas_np = np.cumsum(thetas_np,axis=0)

    def cumsum_flex(arr):
        result = []
        running_sum = 0.0
        for i,x in enumerate(arr):
            running_sum += x
            result.append(running_sum)
        return flex.double(result)

    thetas_flex = cumsum_flex(thetas_flex)

    assert np.all(np.isclose(thetas_np.flatten(),thetas_flex.as_numpy_array()))


    x_np = np.sin(phis_np) * np.cos(thetas_np)
    y_np = np.sin(phis_np) * np.sin(thetas_np)
    z_np = np.cos(phis_np)

    x_flex = flex.sin(phis_flex) * flex.cos(thetas_flex)
    y_flex = flex.sin(phis_flex) * flex.sin(thetas_flex)
    z_flex = flex.cos(phis_flex)


    # Stack x, y, z to form points and multiply by rad
    points_np = rad * np.stack([x_np, y_np, z_np], axis=-1)

    points_flex = rad * flex.vec3_double(x_flex,y_flex,z_flex)
    
    assert np.all(np.isclose(points_np.flatten(),points_flex.as_numpy_array().flatten()))


    # put all together

    # numpy
    points_np = points_np.reshape(1, N, 3)
    # add to ctr
    points_np = ctr_np[:, np.newaxis, :] + points_np


    # flex
    def broadcast_add(ctr_flex, points_flex):
        N = points_flex.size()
        M = ctr_flex.size()
        # Preallocate an array of shape (M, N, 3)
        result = flex.vec3_double(flex.grid(M, N))

        for i in range(M):
            center = ctr_flex[i]
            for j in range(N):
                point = points_flex[j]
                result[i, j] = (center[0] + point[0], center[1] + point[1], center[2] + point[2])

        result = result.as_1d().as_double()
        result.reshape(flex.grid(len(ctr_flex),len(points_flex),3))
        return result
    
    
    points_flex = broadcast_add(ctr_flex,points_flex)

    assert np.all(np.isclose(points_np,points_flex.as_numpy_array()))
    return points_np, points_flex

In [16]:
ctr_np = np.random.random((100,3))
a = sphere_points(ctr_np,2,1000)
b, _ = sphere_points_combined(ctr_np,flex.vec3_double(ctr_np),2,1000)
assert np.all(np.isclose(a,b))

True

In [37]:
floorf_flex(a_flex,2.0*3.6).as_numpy_array()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
pos_quotient.as_numpy_array()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
mod_flex.as_numpy_array()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
import math

a_np = (3.6 / np.sqrt(N * (1.0 - h_np[1:-1]**2))) % (2*math.pi)
a_flex = mod_flex(h_flex,2*math.pi)

In [20]:
floorf_flex(pos_val,divisor)

In [74]:
def __rep(flex_a):
    a = flex_a.as_numpy_array()
    print(a.__class__.__repr__(a))

In [ ]:
__

In [37]:
a = 3.6/flex.sqrt(N * 1.0-h[1:-1]**2)

In [38]:
manual_modulo(a,3.6)

TypeError: unsupported operand type(s) for //: 'double' and 'float'

AttributeError: 'double' object has no attribute '_'

In [36]:
def manual_modulo(a, b):
    # Ensure we don't divide by zero
    if b == 0:
        raise ValueError("The divisor (b) cannot be zero.")
    
    # Calculate the quotient
    q = a // b

    # Calculate the remainder
    remainder = a - (b * q)
    
    return remainder


In [26]:
import math
thetas[1:-1] = (3.6 / flex.sqrt(N * (1.0 - h[1:-1]**2)))

TypeError: 'slice' object is not iterable

In [16]:
 % flex.double(2 * math.pi)

(100,)

In [19]:
h_np.shape

(100, 1)

In [4]:
dm = DataManager()
dm.process_model_file("../data/6cvm.cif")
dm.process_real_map_file("../data/emd_7770.map")
mmm = dm.get_map_model_manager()
assert flex.sum(mmm.map_manager().density_at_sites_cart(mmm.model().get_sites_cart()))>0,(
    "Negative or zero density around atoms. Map/model likely not aligned")
model = mmm.model()
mm = mmm.map_manager()
M = mm.map_data()

In [5]:
from flex_utils import cdist_flex, query_atom_neighbors

ModuleNotFoundError: No module named 'flex_utils'

In [ ]:
inds,dists = query_atom_neighbors(model,radius=3.5)

In [7]:
selection = model.selection("chain A and resseq 1:10").as_numpy_array()

In [8]:
mask_clash=True
voxel_size=1.0
n_probes=8
min_probes=1
radii=np.arange(0.1,2.1,0.1)
rtol=0.9
ignore_min_probes=False
selection_bool=selection
num_processes=1

atoms_xyz = model.get_sites_cart().as_numpy_array()


In [9]:

if selection_bool is None:
      selection_bool = np.full(atoms_xyz.shape[0],True)
atoms_xyz = atoms_xyz[selection_bool]

probe_xyz,keep_mask = radial_shell_mp(atoms_xyz,
                                          n_probes=n_probes,
                                          radii=radii,
                                          rtol = rtol,
                                          num_processes=num_processes)



ValueError: not enough values to unpack (expected 6, got 5)

In [ ]:
# convert to list to have common cctbx/np branch point
n_shells, n_atoms,n_probes,_ = probe_xyz.shape

probe_xyz_l = probe_xyz.tolist()
keep_mask_l = keep_mask.tolist()



# Flatten the nested list to match the expected shape

probe_xyz_l_flat = flatten_nested_list(probe_xyz_l, 3)
keep_mask_l_flat = flatten_nested_list(keep_mask_l,1)

#######################
# cctbx/np branch point
#######################

# TEST LIST CONVERSION

# Convert the flattened nested list back to a numpy array
probe_xyz_l_flat_np = np.array(probe_xyz_l_flat)
keep_mask_l_flat_np = np.array(keep_mask_l_flat)

# Reshape the original numpy array to match the shape of probe_xyz_l_flat_np
reshaped_probe_xyz = probe_xyz.reshape((n_shells*n_atoms*n_probes, 3))
reshaped_keep_mask = keep_mask.reshape((n_shells*n_atoms*n_probes))

# Compare the two arrays
assert np.array_equal(probe_xyz_l_flat_np, reshaped_probe_xyz)
assert np.array_equal(keep_mask_l_flat_np, reshaped_keep_mask)

In [ ]:
%%time
do_np = False
do_flex = True
do_test = False

# to test must do both
if do_test:
    do_np = True
    do_flex = True
# PROCEED



# flatten numpy arrays
if do_np:
    n_shells,n_atoms,n_probes,_ = probe_xyz.shape
    probe_xyz_flat = probe_xyz.reshape((n_atoms*n_shells*n_probes,3))

    keep_mask_flat = keep_mask.reshape(-1) # (n_shells*n_atoms*n_probes,)
    

# init cctbx arrays

if do_flex:
    probe_xyz_cctbx = flex.double(probe_xyz)
    keep_mask_cctbx = flex.bool(keep_mask)




# APPLY MASK BEFORE INTERPOLATION

# numpy
if do_np:
    masked_probe_xyz_flat = probe_xyz_flat[keep_mask_flat]

# cctbx
if do_flex:
    keep_mask_cctbx_fullflat = []

    for val in keep_mask_cctbx:
        for _ in range(3):  # since A has an additional dimension of size 3
            keep_mask_cctbx_fullflat.append(val)

    mask = flex.bool(keep_mask_cctbx_fullflat)
    #indices = flex.int([i for i in range(1, keep_mask_cctbx.size() + 1) for _ in range(3)])
    sel = probe_xyz_cctbx.select(mask)
    #sel_indices = indices.select(mask)
    masked_probe_xyz_flat_cctbx = flex.vec3_double(sel)


# INTERPOLATE

# numpy
if do_np:
    M = mm.map_data()
    volume = M.as_numpy_array()
    voxel_size = np.array(mm.pixel_sizes())
    masked_density = trilinear_interpolation(volume, masked_probe_xyz_flat, voxel_size=voxel_size)

# cctbx
if do_flex:
    masked_density_cctbx = mm.density_at_sites_cart(masked_probe_xyz_flat_cctbx)

    # test equivalent
    if do_test:
        assert np.all(np.isclose(masked_density,masked_density_cctbx.as_numpy_array()))


# reshape interpolated values to (n_shells,n_atoms, n_probes)

# numpy
if do_np:
    d_vals = np.zeros((n_shells, n_atoms, n_probes))
    d_vals[keep_mask] = masked_density

# cctbx
if do_flex:
    keep_mask_cctbx.reshape(flex.grid(n_shells*n_atoms*n_probes))
    d_vals_cctbx = flex.double(keep_mask_cctbx.size(),0.0)
    d_vals_cctbx = d_vals_cctbx.set_selected(keep_mask_cctbx,masked_density_cctbx)
    d_vals_cctbx.reshape(flex.grid(n_shells,n_atoms,n_probes))

    if do_test:
        # test
        assert np.all(np.isclose(d_vals,d_vals_cctbx.as_numpy_array()))


# reshape to (M,N*L) for rowwise correlation

# numpy
if do_np:
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)

# cctbx
if do_flex:
    def custom_reshape_indices(flex_array):
        N,M,L = flex_array.focus()
        result = flex.double(flex.grid(M, N * L))

        for i in range(N):
            for j in range(M):
                for k in range(L):
                    # Calculate the original flat index
                    old_index = i * M * L + j * L + k
                    # Calculate the new flat index after transpose and reshape
                    new_index = j * N * L + i * L + k
                    result[new_index] = flex_array[old_index]

        return result

    d_vals_2d_cctbx = custom_reshape_indices(d_vals_cctbx)


    # test
    if do_test:
        assert np.all(np.isclose(d_vals_2d,d_vals_2d_cctbx.as_numpy_array()))


# create the reference data

# numpy
if do_np:
    M = mm.map_data().as_numpy_array()
    maxD = min(M.mean()+M.std()*10,M.max())
    minD = max(M.mean()-M.std()*1,M.min())
    A = maxD-minD
    B = minD
    u = 0
    sigma = 0.6
    x = radii
    y = A * np.exp(-0.5*((x-u)/sigma)**2) + B 




#cctbx
if do_flex:
    M = mm.map_data()
    maxD_cctbx = min(flex_mean(M)+flex_std(M)*10,flex_max(M))
    minD_cctbx = max(flex_mean(M)-flex_std(M)*1,flex_min(M))
    A_cctbx = maxD_cctbx-minD_cctbx
    B_cctbx = minD_cctbx
    u = 0
    sigma = 0.6
    x = flex.double(radii)
    y_cctbx = A_cctbx * flex.exp(-0.5*((flex.double(x)-u)/sigma)**2) + B_cctbx




    # test
    if do_test:
        assert np.all(np.isclose(np.array(y_cctbx),y))


# Stack and reshape data for correlation calc

# numpy
if do_np:
    # stack the reference to shape (n_shells,n_atoms,n_probes)
    g_vals = np.repeat(y[:,None],n_probes,axis=1)
    g_vals = np.expand_dims(g_vals,1)
    g_vals = np.tile(g_vals,(n_atoms,1))


    # reshape
    g_vals_2d = g_vals.transpose(1,0,2).reshape(g_vals.shape[1], -1)
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)
    mask_2d = keep_mask.transpose(1,0,2).reshape(keep_mask.shape[1], -1)



# cctbx
if do_flex:
    # 1. Repeat y for n_probes (equivalent to np.repeat)
    g_vals_cctbx = [[val] * n_probes for val in y_cctbx]

    # 2. Add a new dimension (equivalent to np.expand_dims)
    g_vals_expanded = [[item] for item in g_vals_cctbx]

    # 3. Tile for each atom (equivalent to np.tile)
    g_vals_tiled = []
    for item in g_vals_expanded:
        g_vals_tiled.append(item * n_atoms)


    g_vals_cctbx = flex.double(np.array(g_vals_tiled) )


    # test 
    if do_test:
        assert np.all(np.isclose(g_vals_cctbx.as_numpy_array(),g_vals))


# # CALCULATE Q

# # numpy
if do_np:
    q = rowwise_corrcoef(g_vals_2d,d_vals_2d,mask=mask_2d)



# cctbx
if do_flex:
    d_vals_cctbx = d_vals_cctbx.as_1d()
    g_vals_cctbx = g_vals_cctbx.as_1d()
    keep_mask_cctbx_double = keep_mask_cctbx.as_1d().as_double()
    q_cctbx = []
    for atomi in range(n_atoms):

        #inds = nd_to_1d_indices((None,atomi,None),(n_shells,n_atoms,n_probes))
        inds = optimized_nd_to_1d_indices(atomi,(n_shells,n_atoms,n_probes))
        inds = flex.uint32(inds)
        d_row = d_vals_cctbx.select(inds)
        
        if do_test:
            assert np.all(np.isclose(d_row.as_numpy_array(),d_vals_2d[atomi]))

        g_row = g_vals_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(g_row.as_numpy_array(),g_vals_2d[atomi]))

        mask = keep_mask_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(mask.as_numpy_array(),mask_2d[atomi]))

        d = d_row.select(mask)
        g = g_row.select(mask)
        qval = flex.linear_correlation(d,g).coefficient()
        q_cctbx.append(qval)
    

    q_cctbx = flex.double(q_cctbx)
    if do_test:
        assert np.all(np.isclose(q,np.array(q_cctbx)))


In [300]:
q_cctbx[:4]

[0.09960622198608986,
 0.29913070189881485,
 0.41570096586160304,
 0.3816933601922859]

In [282]:
print(drow[:4])
print(list(drow_cctbx[:4]))
print(grow[:4])
print(list(grow_cctbx[:4]))

[0.23491911 0.21801951 0.25036227 0.26736135]
[0.2349191050696235, 0.21801951046549, 0.2503622662386241, 0.26736135025163027]
[1.56185174 1.56185174 1.56185174 1.56185174]
[1.561851742665422, 1.561851742665422, 1.561851742665422, 1.561851742665422]


[0.2349191050696235, 0.21801951046549, 0.2503622662386241, 0.26736135025163027]

In [261]:
len(inds)

240

In [138]:
g_vals_2d_cctbx.focus()

(64, 240)

In [129]:
    # do a row
    #shape = (n_atoms,n_shells*n_probes)
    #start,stop =  (i * shape[1], (i+1) * shape[1])
    #g_val = flex.double(flat_list) # the reference
    #d_val = d_vals_2d_cctbx[start:stop]
    #print(list(g_val)[:4])
    #print(list(d_val)[:4])
    #qval = flex.linear_correlation(a,b).coefficient()
    #q_cctbx.append(qval)

-0.09960622198608968

In [108]:
d_vals_2d_cctbx.focus()

(64, 240)

In [89]:
n_shells,n_atoms,n_probes = 20,64,12



20 64 12


In [ ]:
# prepare an output array with zeros
D_3d = np.zeros((N,M,L))
D_3d[C_3d] = D
D_2d = D_3d.transpose(1,0,2).reshape(D_3d.shape[1], -1)

In [104]:
# prepare an output array with zeros
d_vals = np.zeros((n_shells, n_atoms, n_probes))

d_vals[keep_mask] = masked_density
d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)

# cctbx
def reshape_density_to_full(masked_density_cctbx,keep_mask_flat_cctbx,n_shells,n_atoms,n_probes):
    N,M,L = n_shells, n_atoms, n_probes
    C = keep_mask_flat_cctbx
    D = masked_density_cctbx
    
    result = []
    index_d = 0
    for i in range(N):
        result_m = []
        for j in range(M):
            result_l = []
            for k in range(L):
                # Calculate the equivalent flat index for C
                flat_index = i * M * L + j * L + k
                if C[flat_index]:
                    result_l.append(D[index_d])
                    index_d += 1
                else:
                    result_l.append(fill_value)
            result_m.append(result_l)
        result.append(result_m)
    return result

def reshape_density_to_rows(d_vals_cctbx,n_shells,n_atoms,n_probes):
    N,M,L = n_shells, n_atoms, n_probes
    
    
    D_3d = d_vals_cctbx
    result = []
    for j in range(M):
        D_2d_row = []
        for i in range(N):
            for k in range(L):
                D_2d_row.append(D_3d[i][j][k])
        result.append(D_2d_row)
    return result

# test full reshape to (n_shells,n_atoms,n_probes)
d_vals_cctbx = reshape_density_to_full(masked_density_cctbx,
                                      keep_mask_flat_cctbx,
                                      n_shells,n_atoms,n_probes)

assert np.all(np.isclose(np.array(d_vals_cctbx),d_vals))

# test reshape to rows (n_atoms,n_shells*n_probes)
d_vals_2d_cctbx = reshape_density_to_rows(d_vals_cctbx,n_shells,n_atoms,n_probes)

assert np.all(np.isclose(np.array(d_vals_2d_cctbx),d_vals_2d))

In [111]:
fill_value = 0.0  # or any value you choose as the fill value
result = [[] for _ in range(M)]
index_d = 0

for n in range(N):
    for m in range(M):
        l_start,l_end = 0,L
        for l in range(L):
            # Calculate the equivalent flat index for C
            flat_index = n * M * L + m * L + l
            
            # Determine the value to append
            if C[flat_index]:
                value_to_append = D[index_d]
                index_d += 1
            else:
                value_to_append = fill_value

            # Append directly to the appropriate row in the result
            result[j].extend([value_to_append])

In [122]:
d_vals_cctbx_flat = flex.double(n_shells*n_atoms*n_probes,0.0)
d_vals_cctbx_flat = d_vals_cctbx_flat.set_selected(keep_mask_flat_cctbx,masked_density_cctbx)

In [173]:
from itertools import chain
d_vals_2d_cctbx = []
indices_atoms = []
for m in range(M):
    row_indices = []
    for n in range(N):
        l_start = 0
        l_end = L-1
        
        flat_start = n * M * L + m * L + l_start
        flat_end = n * M * L + m * L + l_start
        l_chunk_indices = list(range(flat_start,flat_end))
        row_indices.append(l_chunk_indices)
    indices_atoms.append(list(chain.from_iterable(row_indices)))
        
# d_vals_2d_cctbx = []
# for row in indices_atoms:
#     chunk = d_vals_cctbx_flat.select(row)
#     d_vals_2d_cctbx.append(chunk)

In [181]:
a = flex.double(d_vals)
print(a.focus())

(20, 64, 12)


In [182]:
a.select(masked_density)

TypeError: No registered converter was able to produce a C++ rvalue of type unsigned int from this Python object of type numpy.float64

In [170]:
np.isclose(np.array(d_vals_2d_cctbx),d_vals_2d)

ValueError: operands could not be broadcast together with shapes (64,220) (64,240) 

In [171]:
np.array(d_vals_2d_cctbx).shape

(64, 220)

In [149]:
d_vals_cctbx_flat.select??

Docstring:
select( (double)self, (bool)flags) -> double :

    C++ signature :
        scitbx::af::shared<double> select(scitbx::af::versa<double, scitbx::af::flex_grid<scitbx::af::small<long, 10ul> > >,scitbx::af::const_ref<bool, scitbx::af::trivial_accessor>)

select( (double)self, (object)indices [, (object)reverse=False]) -> double :

    C++ signature :
        scitbx::af::shared<double> select(scitbx::af::versa<double, scitbx::af::flex_grid<scitbx::af::small<long, 10ul> > >,scitbx::af::const_ref<unsigned int, scitbx::af::trivial_accessor> [,bool=False])

select( (double)self, (size_t)indices [, (object)reverse=False]) -> double :

    C++ signature :
        scitbx::af::shared<double> select(scitbx::af::versa<double, scitbx::af::flex_grid<scitbx::af::small<long, 10ul> > >,scitbx::af::const_ref<unsigned long, scitbx::af::trivial_accessor> [,bool=False])

select( (double)arg1, (unsigned)selection) -> double :

    C++ signature :
        scitbx::af::shared<double> select(scitbx::af

In [142]:
np.array(d_vals_2d_cctbx).shape

(1280, 11)

In [31]:
# array A is shape (N,M,L)
B = A.reshape(-1)

# array C.shape==B.shape (N*M*L). C is a boolean mask

# select values
D = B[C]



# I now want to transform D to list of len N*M where each value is a list of len L, where the values are taken from D where there were selected (C==True), or are a fill value if they correspond to a False value in C

array([ 0.23491911,  0.21801951,  0.25036227, ..., -0.17700099,
       -0.21437779, -0.20423411])

In [15]:


M = mm.map_data()
maxD = min(flex_mean(M)+flex_std(M)*10,flex_min(M))
minD = max(flex_mean(M)-flex_std(M)*1,flex_min(M))
A = maxD-minD
B = minD
u = 0
sigma = 0.6
x = radii
y = A * np.exp(-0.5*((x-u)/sigma)**2) + B

In [ ]:
flex.v

In [ ]:
def Qscore(volume,
                atoms_xyz,
                mask_clash=True,
                voxel_size=1.0,
                n_probes=8,
                min_probes=1,
                radii=np.arange(0.1,2.1,0.1),
                rtol=0.9,
                ignore_min_probes=False,
                selection_bool=None,
                num_processes=cpu_count()):

    # handle selection at the very beginning
    if selection_bool is None:
      selection_bool = np.full(atoms_xyz.shape[0],True)
    atoms_xyz = atoms_xyz[selection_bool]


              
    probe_xyz,keep_mask = radial_shell_mp(atoms_xyz,
                                          n_probes=n_probes,
                                          radii=radii,
                                          rtol = rtol,
                                          num_processes=num_processes)
    
    n_shells,n_atoms,n_probes,_ = probe_xyz.shape
    probe_xyz_flat = probe_xyz.reshape((n_atoms*n_shells*n_probes,3))
    keep_mask_flat = keep_mask.reshape(-1) # (n_shells*n_atoms*n_probes,)
    
    # apply mask to the flattened probe_xyz
    masked_probe_xyz_flat = probe_xyz_flat[keep_mask_flat]
    #masked_probe_xyz_flat_flex = flex.vec3_double(masked_probe_xyz_flat)
    
    # apply trilinear interpolation only to the relevant probes
    masked_density = trilinear_interpolation(volume, masked_probe_xyz_flat, voxel_size=voxel_size) # (n_valid_probes,)
    #masked_density = mm.density_at_sites_cart(masked_probe_xyz_flat_flex).as_numpy_array()
    
    # prepare an output array with zeros
    d_vals = np.zeros((n_shells, n_atoms, n_probes))
    
    # reshape interpolated values to (n_shells, n_atoms, n_probes) using the mask
    d_vals[keep_mask] = masked_density
    
    
    
    n_atoms = probe_xyz.shape[1]
    n_probes = probe_xyz.shape[2]
    M = volume
    maxD = min(M.mean()+M.std()*10,M.max())
    minD = max(M.mean()-M.std()*1,M.min())
    A = maxD-minD
    B = minD
    u = 0
    sigma = 0.6
    x = radii
    y = A * np.exp(-0.5*((x-u)/sigma)**2) + B 
    
    # stack the reference to shape (n_shells,n_atoms,n_probes)
    g_vals = np.repeat(y[:,None],n_probes,axis=1)
    x_repeat = np.repeat(x,n_probes)
    g_vals = np.expand_dims(g_vals,1)
    
    g_vals = np.tile(g_vals,(n_atoms,1))
    
    # Reshape to 2d for masked rowwise correlation calculation
    g_vals_2d = g_vals.transpose(1,0,2).reshape(g_vals.shape[1], -1)
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)
    mask_2d = keep_mask.transpose(1,0,2).reshape(keep_mask.shape[1], -1)
    
    q = rowwise_corrcoef(g_vals_2d,d_vals_2d,mask=mask_2d)
    return q,probe_xyz,keep_mask,d_vals, g_vals